# Tarefas Módulo 8
- Nome: Cícero Henrique dos Santos
- Data: XX/10/2022


### Considere o dataset abaixo:

In [36]:
import pandas as pd
df = pd.read_csv("dataframe_exercicio_modulo_8.csv")
print(df.shape)
df.head()

(2460, 5)


,ind_pop_cid,perc_pol,taxa_crim,nivel_ed,crimes
0,1.0,7.185471,62.082693,10.0,77
1,1.0,24.456750,88.244380,6.0,202
2,1.0,19.791772,94.930257,1.0,348
3,4.0,10.089496,41.791507,6.0,114
4,3.0,34.697495,67.615795,1.0,47


Descrição das variáveis:

- ind_pop_cid = índice entre 1 e 5, indicando o tamanho da população de uma cidade; quanto maior o índice, maior a cidade;

- perc_pol = percentual de policiais ativos na cidade;

- taxa_crim = taxa de criminalidade histórica da cidade;

- nivel_ed = nível educacional médio da cidade;

- crimes = quantidade de crimes registrados no mês vigente;


# Exercícios

## Parte 1:

Os exercícios abaixo tratarão a respeito da criação de modelos de aprendizado de máquina. Vimos que precisamos controlar o overfitting dos modelos e medir a capacidade de generalização dos mesmos. Para isso, é interessante termos os dados disponíveis divididos em treino/validação/teste. 


#### Crie uma função para fazer a divisão de treino/teste.

Esta função deve ter dois parâmetros de entrada:

    1- dataframe disponível;
    2- percentual do dataset que será disponibilizado para os dados de teste;
    
e deve retornar dois dataframes.

(__Exemplo:__ Se o dataframe disponível possui 100 linhas e queremos 25% dos dados para teste, então a função deve retornar dois dataframes, um com 75 linhas (o dataset de treino) e outro com as 25 linhas restantes (o dataset de teste).)

In [57]:
def divide_teste_e_treino(dataframe, percentual_teste: float):
    
    # Copia o dataframe base
    df = dataframe.copy()
    
    # Com base na porcentagem, identifica a linha de corte de acordo com o total de quantidade de linhas do dataframe
    qtd_dados_teste = len(df)*(percentual_teste/100.00)
    qtd_dados_teste = int(qtd_dados_teste) # Transforma em numero inteiro
    
    # Criar os dataframes de teste e treino com base na porcentagem passada 
    df_teste = pd.DataFrame(df.iloc[:qtd_dados_teste])
    df_treino = pd.DataFrame(df.iloc[qtd_dados_teste:])
    
    # Returna os dataframes de treino e teste
    return df_teste, df_treino

In [65]:
# Usa a função para dividir a porcentagem de linhas de treino e teste de um dataframe
divide_teste_e_treino(df, 25.0)

(     ind_pop_cid   perc_pol  taxa_crim  nivel_ed  crimes
 0            1.0   7.185471  62.082693      10.0      77
 1            1.0  24.456750  88.244380       6.0     202
 2            1.0  19.791772  94.930257       1.0     348
 3            4.0  10.089496  41.791507       6.0     114
 4            3.0  34.697495  67.615795       1.0      47
 ..           ...        ...        ...       ...     ...
 610          3.0   6.987886  64.352064       3.0     344
 611          4.0   4.091551  52.447571       4.0     253
 612          2.0   1.484661  36.890828       3.0     139
 613          4.0  17.015379  61.276596       9.0     148
 614          2.0  42.683878  93.438586       3.0      12
 
 [615 rows x 5 columns],
       ind_pop_cid   perc_pol  taxa_crim  nivel_ed  crimes
 615           4.0  28.105493  78.875703       7.0     127
 616           4.0  21.987137  92.904009       8.0     276
 617           4.0  16.073622  23.303663       1.0      29
 618           4.0  12.529803  35.832769 

In [66]:
# Cria 2 dataframes com base no retorno da função 
df_teste, df_treino = divide_teste_e_treino(df, 25.0)

In [70]:
# Reseta o index no dataframe de teste
df_teste.reset_index()

,index,ind_pop_cid,perc_pol,taxa_crim,nivel_ed,crimes
0,0,1.0,7.185471,62.082693,10.0,77
1,1,1.0,24.456750,88.244380,6.0,202
2,2,1.0,19.791772,94.930257,1.0,348
3,3,4.0,10.089496,41.791507,6.0,114
4,4,3.0,34.697495,67.615795,1.0,47
...,...,...,...,...,...,...
610,610,3.0,6.987886,64.352064,3.0,344
611,611,4.0,4.091551,52.447571,4.0,253
612,612,2.0,1.484661,36.890828,3.0,139
613,613,4.0,17.015379,61.276596,9.0,148


In [69]:
# Reseta o index no dataframe de treino
df_treino.reset_index()

,index,ind_pop_cid,perc_pol,taxa_crim,nivel_ed,crimes
0,615,4.0,28.105493,78.875703,7.0,127
1,616,4.0,21.987137,92.904009,8.0,276
2,617,4.0,16.073622,23.303663,1.0,29
3,618,4.0,12.529803,35.832769,4.0,84
4,619,4.0,11.856335,35.529906,7.0,53
...,...,...,...,...,...,...
1840,2455,1.0,3.507206,56.070224,1.0,231
1841,2456,4.0,28.222931,46.858543,3.0,18
1842,2457,1.0,13.055995,87.372664,9.0,217
1843,2458,2.0,22.632517,46.170517,6.0,42


---

## Parte 2:

### 1 - Usando __apenas UMA variável__, crie modelos de aprendizado de máquina para prever a quantidade de crimes a partir desta única variável. Justifique a escolha desta variável. Instancie vários modelos e compare-os de acordo com as métricas de performance. Indique quais modelos estão sofrendo overfitting. 

### 2 - Agora, usando todas as variáveis disponíveis, refaça as análises do primeiro exercício e compare os resultados.
